In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils_nlp.models.pretrained_embeddings.fasttext import load_pretrained_vectors
from utils_nlp.dataset.muse import load_pandas_df
from utils_nlp.models.geomm.utils import LangEdge, LangMapping, evaluate_lang_pair, create_train_mapping, create_dictionary
from utils_nlp.models.geomm.geomm import get_geomm_mapping
import numpy as np
import collections

unable to import 'smart_open.gcs', disabling that module
/data_drive/mayank/anaconda3/envs/nlp_gpu/lib/python3.6/site-packages/dask/dataframe/utils.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
BASE_DATA_PATH = '/home/mayank/data/nlp-recipes/'

In [4]:
src_label = 'en'
trg_label = 'es'

In [5]:
embeddings = {}
embeddings[src_label] = load_pretrained_vectors(BASE_DATA_PATH + 'embedding', file_name=f"wiki.{src_label}.vec", limit=200000)
embeddings[trg_label] = load_pretrained_vectors(BASE_DATA_PATH + 'embedding', file_name=f"wiki.{trg_label}.vec", limit=200000)

Vector file already exists. No changes made. /home/mayank/data/nlp-recipes/embedding/fastText/wiki.en.vec
Vector file already exists. No changes made. /home/mayank/data/nlp-recipes/embedding/fastText/wiki.es.vec


In [6]:
pairs_data_train = load_pandas_df(local_cache_path=BASE_DATA_PATH + 'muse', file_split='train', src_language=src_label, trg_language=trg_label)

In [7]:
pairs_data_test = load_pandas_df(local_cache_path=BASE_DATA_PATH + 'muse', file_split='test', src_language=src_label, trg_language=trg_label)

In [8]:
train_mapping = create_train_mapping(pairs_data_train, embeddings[src_label], embeddings[trg_label])

In [9]:
edge = LangEdge(src_label, trg_label, mapping=train_mapping)

In [10]:
import tensorflow as tf

In [20]:
geomm_result = get_geomm_mapping([edge], embeddings, maxiter=200, lr=1e2, verbosity=3, tf_dtype=tf.float64)

Compiling cost function...
Computing gradient of cost function...
Optimizing...
 iter		   cost val	    grad. norm
    0	+3.7328301919552542e+07	5.78646380e+07
    1	+3.5458651052430794e+07	2.27435622e+07
    2	+3.4390953273104593e+07	1.96986944e+07
    3	+3.2954150361525204e+07	1.85899686e+07
    4	+3.1587167762439702e+07	2.24073744e+07
    5	+2.9053652584775385e+07	1.10362143e+07
    6	+2.8013195601504192e+07	1.54854290e+07
    7	+2.5589212404718395e+07	1.12107024e+07
    8	+2.4406451209357396e+07	1.12261460e+07
    9	+2.2375070610780582e+07	8.38826628e+06
   10	+2.1303657238065992e+07	8.28507144e+06
   11	+2.0090034976637237e+07	3.95160100e+06
   12	+1.9621217747885600e+07	3.56091680e+06
   13	+1.8866327911963642e+07	3.41423089e+06
   14	+1.7502463590875838e+07	7.03938356e+06
   15	+1.4519829257680617e+07	5.72403734e+06
   16	+1.2943155256194429e+07	6.35105422e+06
   17	+1.0827082056073653e+07	2.11799754e+06
   18	+1.0426142553170441e+07	1.61661251e+06
   19	+9.8082913889889680e+06	1

  180	-2.0161105744062792e+02	3.65077741e+00
  181	-2.0180413816573176e+02	3.38937045e+00
  182	-2.0196940178053396e+02	3.05784205e+00
  183	-2.0210781221980579e+02	2.82196141e+00
  184	-2.0222638197846740e+02	2.52196180e+00
  185	-2.0232490031314674e+02	2.27862715e+00
  186	-2.0240466158796576e+02	2.14123189e+00
  187	-2.0247587011620698e+02	2.14593162e+00
  188	-2.0255068795774343e+02	2.06569050e+00
  189	-2.0262460630799583e+02	2.08358328e+00
  190	-2.0270888088101003e+02	2.01883997e+00
  191	-2.0279603862736272e+02	1.91062282e+00
  192	-2.0287894472634844e+02	1.80248724e+00
  193	-2.0295594557517833e+02	1.69831974e+00
  194	-2.0306621894373868e+02	3.62335460e+00
  195	-2.0320695877335089e+02	3.63529559e+00
  196	-2.0329304738927962e+02	2.38498660e+00
  197	-2.0332049783584037e+02	1.72946870e+00
  198	-2.0333770747340793e+02	1.37858000e+00
  199	-2.0335757546386228e+02	1.35873780e+00
Terminated - max iterations reached after 16.78 seconds.



In [21]:
test_dictionary = create_dictionary(pairs_data_test, embeddings[src_label], embeddings[trg_label])

In [22]:
evaluate_lang_pair(embeddings[src_label].vectors, embeddings[trg_label].vectors, geomm_result[1][src_label], geomm_result[1][trg_label], geomm_result[0], test_dictionary)

100%|██████████| 2/2 [00:06<00:00,  4.15s/it]


{'Cosine Accuracy': 0.7711598746081505, 'CSLS Accuracy': 0.7711598746081505}